In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user


In [ ]:
from pyspark import SparkContext

In [ ]:
spark.stop()

In [ ]:
sc = pyspark.SparkContext(appName="performanceFactors")

In [ ]:
factors = sc.textFile("/content/StudentPerformanceFactors.csv")

In [ ]:
factors.getNumPartitions()

2

In [ ]:
print(type(factors))

<class 'pyspark.rdd.RDD'>


In [ ]:
print(factors.getNumPartitions())

2


In [10]:
!pip install pyspark
from pyspark import SparkContext
sc = SparkContext(appName="performanceFactors")
factorsrdd = sc.textFile("/content/StudentPerformanceFactors.csv")

In [20]:
header = factorsrdd.first()  # header
datardd = factorsrdd.filter(lambda row: row != header).map(lambda line: line.split(","))  # rdd

column_names = header.split(",")  # split headers
column_index = {column_names[i].strip(): i for i in range(len(column_names))}  # dictionary for each column

# index of specific columns
exam_score_column = column_index['Exam_Score']
previous_exam_column = column_index['Previous_Scores']
sleep_column = column_index['Sleep_Hours']
hrs_studied_column = column_index['Hours_Studied']
income_column = column_index['Family_Income']
Access_to_Resources = column_index['Access_to_Resources']
Teacher_column = column_index['Teacher_Quality']

# Filter students who passed both exams
passed_both_filtered = datardd.filter(lambda row: (
    int(row[previous_exam_column].strip()) >= 70 and
    int(row[exam_score_column].strip()) >= 70
))

# Filter students who passed the previous exam but not the recent one
passed_previous_filtered = datardd.filter(lambda row: (
    int(row[previous_exam_column].strip()) >= 70 and
    int(row[exam_score_column].strip()) < 70
))

# Combine selected columns
both_exam = passed_both_filtered.map(lambda row: (
    int(row[hrs_studied_column].strip()),
    (int(row[previous_exam_column].strip()),
     row[income_column].strip(),
     row[sleep_column].strip() + " hrs",
     int(row[exam_score_column].strip()))
))

previous_exam = passed_previous_filtered.map(lambda row: (
    row[income_column].strip(),
    (int(row[previous_exam_column].strip()),
     int(row[hrs_studied_column].strip()),
     row[sleep_column].strip() + " hrs",
     int(row[exam_score_column].strip()))
))

# Group by Family Income and sort by Previous Exam Score within each group
groupby_income = previous_exam.groupByKey()
sortedby_income = groupby_income.mapValues(lambda records: sorted(records, key=lambda x: x[1]))

# Collect and print the results
groupsorted_data = sortedby_income.collect()

#for group_income, records in groupsorted_data:
#    print("Family Income: ", group_income)
#    for record in records:
#        print(f"  Hours_Studied: {record[1]}, Previous_Score: {record[0]}, Sleep_Hours: {record[2]}, Recent_Exam_Score: {record[3]}")


In [9]:
sc.stop()

In [33]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import col, when, count

spark = SparkSession.builder.appName("Student Performance").getOrCreate()

df = datardd.map(lambda row: Row(
    Hours_Studied=int(row[hrs_studied_column].strip()),
    Previous_Scores=int(row[previous_exam_column].strip()),
    Family_Income=row[income_column].strip(),
    Sleep_Hours=int(row[sleep_column].strip()),
    Exam_Score=int(row[exam_score_column].strip()),
    Resources=row[Access_to_Resources].strip(),
    Teacher_Quality=row[Teacher_column].strip()
)).toDF()

# select
selected_df = df.select("Hours_Studied", "Previous_Scores", "Family_Income", "Sleep_Hours", "Exam_Score", "Resources", "Teacher_Quality")
# filter
filtered_df = selected_df.filter(selected_df["Previous_Scores"] >= 70)

# adding score range
score_range_df = filtered_df.withColumn(
    "Score_Range",
    when((col("Previous_Scores") >= 90), "Line of 9 and above")
    .when((col("Previous_Scores") >= 80) & (col("Previous_Scores") < 90), "Line of 8")
    .when((col("Previous_Scores") >= 70) & (col("Previous_Scores") < 80), "Line of 7")
)

grouped_score_income_df = score_range_df.groupBy("Resources", "Score_Range").agg(count("*").alias("Total_Students"))

# final results
grouped_score_income_df.orderBy("Resources", "Score_Range").show()

+---------+-------------------+--------------+
|Resources|        Score_Range|Total_Students|
+---------+-------------------+--------------+
|     High|          Line of 7|           395|
|     High|          Line of 8|           412|
|     High|Line of 9 and above|           427|
|      Low|          Line of 7|           271|
|      Low|          Line of 8|           258|
|      Low|Line of 9 and above|           243|
|   Medium|          Line of 7|           660|
|   Medium|          Line of 8|           654|
|   Medium|Line of 9 and above|           721|
+---------+-------------------+--------------+

